# Техническое задание 1

**Распознавание текста с картинки в формате jpg и тестирование автокорректоров.**

Сделать небольшой обзор на автокорректоры:
1. Взять модель tesseract, которая распознает русский текст
2. Распознать любой текст (10 страниц любого литературного текста на русском языке без формул, цифры разрешены) и посчитать метрики. Использовать метрики WER и CER.
3. Взять любой уже предобученный автокорректор и замените слова. И снова сравните с ошибками.
4. В ходе работы задание дополнено: протестировать небольшой текст из любого не математического учебника (более академический слог).

## План работы:
### 1. Подготовка данных и инструментов

**Сбор данных:**

Найти текст 10 страниц в формате djvu на русском языке.
Найти этот же текст в формате txt, чтобы можно было сравнивать с распознанным текстом.

**Подготовка инструментов**

Установить модель tesseract для распознавания текстов. Подготовить и установить автокорректоры.

### 2. Распознавание текста с помощью Tesseract
**Загрузка и обработка изображения:**

Загрузить изображение текста. Применить предварительную обработку изображения, если необходимо (например, улучшение контраста, удаление шума).

**Распознавание текста:**

Использовать pytesseract для преобразования изображения в текст.
Сохранить распознанный текст в один файл.

### 3. Анализ ошибок:

Сравнить исходный текст с распознанным текстом.
Посчитать количество ошибок (пропущенных символов, неверных символов и т.п.) используя метрики WER и CER.

### 4. Применение автокорректора

**Применение автокорректора:**

Взять распознанный текст и применить к нему выбранную модель автокорректора.
Сохранить исправленный текст в отдельную переменную.

**Оценка результатов:**

Сравнить исходный текст с исправленным текстом.
Оценить количество оставшихся ошибок после применения автокорректора, используя метрики WER и CER.

### 5. Анализ и выводы
Провести анализ полученных результатов и сделать выводы о качестве работы автокорректоров.

### 1. Подготовка данных и инструментов

**Установка библиотек**

In [1]:
pip install deeppavlov # иногда подключается за два захода

In [2]:
!pip install pytesseract
!pip install Pillow

In [3]:
# Установка Tesseract
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-rus  # для поддержки русского языка

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-rus is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
pip install pytesseract pillow jiwer # установка библиотеки для вычисления метрик

In [5]:
pip install pyspellchecker # библиотеки авторорректоров

In [6]:
!apt-get update
!apt-get install -y libhunspell-dev hunspell-ru
!pip install hunspell

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [7]:
pip install pytesseract autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=d19fc9951661bd343132338f66d70f9950ea2feee995b6d32b849e7abc8384fd
  Stored in directory: /root/.cache/pip/wheels/5e/90/99/807a5ad861ce5d22c3c299a11df8cba9f31524f23ae6e645cb
Successfully built autocorrect


In [8]:
!pip install sacremoses # необходим для deeppavlov

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.9 MB/s eta 0:00:00


In [9]:
pip install sortedcontainers # необходим для deeppavlov

In [10]:
pip install kenlm # необходим для deeppavlov

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.3.0-cp311-cp311-linux_x86_64.whl size=3185139 sha256=563019e2f2130bf84783839c5e5396ead9d08f1b462e13ba1655e1ab1021da2d
  Stored in directory: /root/.cache/pip/wheels/eb/fd/7c/6630a05c6fe34e23f266b8156e4c585a5ae487f8fd7c04a3ef
Successfully built kenlm


In [11]:
import pandas as pd
import pytesseract
import os
import requests
import subprocess
import re
import string
import jiwer # библиотека для вычисления метрик
from PIL import Image, ImageFilter
from io import BytesIO

In [12]:
from spellchecker import SpellChecker
import hunspell
from autocorrect import Speller
from deeppavlov import build_model, configs


**Сбор данных:**

Подготовила **текст для распознавания:** отрывок из книги братьев Стругацких "Понедельник начинается в субботу" в формате djvu. Конвертировала их с стороннем сервисе в файл jpg. Согласно заданию - это чистый литературный текст с небольшим количеством цифр. Текст был взят из середины книги, старалась выбрать часть с наименьшим количеством прямой речи. В тексте есть переносы, это необходимо было учитывать в работе.

Подготовила **эталонный файл:** вырезала эту же часть текста из файла в формате txt. Предварительно проверила на предмет опечаток и ошибок.

Для сравнения был добавлен и распознан текст из учебника "Теория государства и права" для того чтобы убедиться, что на результат могут влиять специфические слова из литературного текста.

### 2. Распознавание текста с помощью Tesseract

**Литературный текст**

In [13]:
from google.colab import drive # подключение гугл диска с файлами
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# путь к папке с изображениями
image_folder_path = '/content/drive/My Drive/Files_for_project/images'

# список всех файлов изображений в указанной папке
image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.jpg',
                                                                       '.jpeg',
                                                                       '.png'))]

image_files.sort()

# распознавание и объединение текста из всех изображений
combined_text = ""

for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    img = Image.open(image_path)
    img = img.convert('L')  # преобразование в оттенки серого
    img = img.filter(ImageFilter.MedianFilter())  # удаление шума
    threshold = 128
    img = img.point(lambda p: p > threshold and 255)  # бинаризация
    text = pytesseract.image_to_string(img, lang='rus')  # распознавание
    combined_text += text + "\n"

print("Часть объединенного текста:")
print(combined_text[0:2550]) # печать части текста, для проверки

Часть объединенного текста:
по-видимому, с вестибюля. В заплесневелых углах белесо мер-
цали кости прикованных скелетов, где-то мерно капала вода,
в нишах между колоннами в неестественных позах торчали
статуи в ржавых латах, справа от входа у стены громоздились
обломки древних идолов, наверху этой кучи торчали гипсо-
вые ноги в сапогах. С почерневших портретов под потолком
строго взирали маститые старцы, в их лицах усматривались
знакомые черты Федора Симеоновича, товарища Жиана Жиа-
комо и других мастеров. Весь этот архаический хлам надле-
жало давным-давно выбросить, прорубить в стенах окна и
установить трубки дневного света, но все было заприходовано,
заинвентаризовано и лично Модестом Матвеевичем к разба-
зариванию запрещено.

На капителях колонн и в лабиринтах исполинской люстры,
свисающей с почерневшего потолка, шуршали нетопыри и ле-
тучие собаки. С ними Модест Матвеевич боролся. Он поливал
их скипидаром и креозотом, опылял дустом, опрыскивал гек-
сахлораном, они гибли тысячами, 

In [15]:
def remove_hyphens(text):
    # Убираем переносы слов, заменяя '-\n' на пустую строку
    text = re.sub(r'-\n', '', text)
    # Убираем переносы строк, если они не следуют за переносом слова
    text = re.sub(r'\n', ' ', text)
    return text

In [16]:
recognized_text = remove_hyphens(combined_text) # убираем переносы
print(recognized_text[0:2550]) # печать части текста, для проверки

по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятками тысяч. Они мутир

In [17]:
# Сохранение текста в файл
with open("rec_lit_text.txt", "w") as file:
    file.write(recognized_text)

In [18]:
# загрузим эталонный файл
etalon_file = '/content/drive/My Drive/Files_for_project/Sample.txt'
etalon_text = open(etalon_file, 'r', encoding='utf-8').read()
print(etalon_text[0:2550])

﻿по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризировано и лично Модестом Матвеевичем к разбазариванию запрещено.
На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятками тысяч. Они мут

In [19]:
etalon_text = remove_hyphens(etalon_text) # удалим переносы строки
print(etalon_text[0:2550])

﻿по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризировано и лично Модестом Матвеевичем к разбазариванию запрещено. На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятками тысяч. Они мут

In [20]:
# Сохранение текста в файл
with open("etalon_lit_text.txt", "w") as file:
    file.write(etalon_text)

**Текст из учебника**

In [21]:
# путь к папке с изображениями
image_folder_path = '/content/drive/My Drive/Files_for_project/images_textbook_3'

# список всех файлов изображений в указанной папке
image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.jpg',
                                                                       '.jpeg',
                                                                       '.png'))]

image_files.sort()

# распознавание и объединение текста из всех изображений
combined_text = ""

for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    img = Image.open(image_path)
    img = img.convert('L')  # преобразование в оттенки серого
    img = img.filter(ImageFilter.MedianFilter())  # удаление шума
    threshold = 128
    img = img.point(lambda p: p > threshold and 255)  # бинаризация
    text = pytesseract.image_to_string(img, lang='rus')  # распознавание
    combined_text += text + "\n"

print("Часть объединенного текста:")
print(combined_text[0:1000]) # печать части текста, для проверки

Часть объединенного текста:
Субъектом правопонимания всегда выступает конкрет-
ный человек, например: а) гражданин, обладающий мини-
мальным правовым кругозором, столкнувшийся с пробле-
мами права вообще; 6) юрист-профессионал, имеющий
достаточный запас знаний о праве, способный применять
и толковать правовые нормы; в) ученый, человек с аб-
страктным мышлением, занимающийся изучением права,
обладающий суммой исторических и современных знаний,
способный к интерпретации не только норм, но и прин-
ципов права, владеющий определенной методологией ис-
следования. Правопонимание всегда субъективно, ори-
гинально, хотя представления о праве могут совпадать
у группы лиц и у целых слоев, классов.

Объектом правопонимания могут быть право в плане-
тарном масштабе, право конкретного общества, отрасль,
институт права, отдельные правовые нормы. При этом
знания об отдельных структурных элементах экстрапо-
лируются на право в целом. Важную познавательную на-
грузку здесь несут среда и взаимодействующ

In [22]:
# убираем переносы слов и строк
recognized_text_tgp = remove_hyphens(combined_text)
print(recognized_text_tgp[0:2550]) # печать части текста, для проверки

Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления.  Содержание пр

In [23]:
# Сохранение текста в файл
with open("rec_educ_text.txt", "w") as file:
    file.write(recognized_text_tgp)

In [24]:
# загрузим эталонный файл
etalon_file_tgp = '/content/drive/My Drive/Files_for_project/tgp_3.txt'
etalon_text_tgp = open(etalon_file_tgp, 'r', encoding='utf-8').read()
etalon_text_tgp = remove_hyphens(etalon_text_tgp) # удалим переносы строки
print(etalon_text_tgp[0:2550])

﻿Субъектом правопонимания всегда выступает конкретный человек, например:  а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще;  б) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы;  в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования.  Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов. Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. Содержание

Видно, что есть лишние пробелы. Удалим их сразу.

In [25]:
def remove_extra_spaces(text):
  """
  Удаляет лишние пробелы из текста, оставляя только один пробел между словами.
  """
  return ' '.join(text.split())

In [26]:
# в эталонном тексте видно лишние пробелы, удалим
etalon_text_tgp = remove_extra_spaces(etalon_text_tgp)
print(etalon_text_tgp[0:1000])

﻿Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; б) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов. Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. Содержание пра

In [27]:
# Сохранение текста в файл
with open("etalon_educ_text.txt", "w") as file:
    file.write(etalon_text_tgp)

**Результат:**

Файлы в формате jpg открыты, обработаны для лучшей распознаваемости, распознаны моделью tesseract и объединены в один файл. Новый сформированный текст очищен от переносов и знаков переноса.

Открыт эталонный файл в формате txt. Удалены переносы строки. Тексты готовы к дальнейшей работе.

### 3. Анализ ошибок:

Для улучшения точности вычисления CER рекомендуется предобрабатывать тексты: приводить их к нижнему регистру и удалять лишние символы, такие как пунктуация.

In [29]:
def preprocess_text(text):
    """
    Функция для обработки текста для CER
    Приводит текст к нижнему регистру и удаляет знаки препинания и пробелы.
    """
    return ''.join(c.lower() for c in text if c not in string.punctuation and not c.isspace())

In [89]:
def metrics_wer_cer(etalon_text, recognized_text):
    """
    Вычисление метрик CER и WER
    """
    # Обработка текстов для CER
    etalon_text_prep = preprocess_text(etalon_text)
    recognized_text_prep = preprocess_text(recognized_text)

    # Вычисление CER и WER
    wer = jiwer.wer(etalon_text, recognized_text)
    cer = jiwer.cer(etalon_text_prep, recognized_text_prep)

    return round(wer, 3), round(cer, 3)

**Литературный текст**

In [90]:
wer_recognized, cer_recognized = metrics_wer_cer(etalon_text, recognized_text)
print(f"WER: {wer_recognized}")
print(f"CER: {cer_recognized}")

WER: 0.046
CER: 0.004


**Текст из учебника**

In [91]:
wer_recognized_tgp, cer_recognized_tgp = metrics_wer_cer(etalon_text_tgp,
                                                         recognized_text_tgp)
print(f"WER: {wer_recognized_tgp}")
print(f"CER: {cer_recognized_tgp}")

WER: 0.023
CER: 0.002


**Результат:**

Метрики для литературных текстов без применения автокоррекции равны WER: 4,6% и CER: 0,4%.

Метрики для текста из учебника без применения автокоррекции равны WER: 2,3% и CER: 0,2%. Метрики литературного текста в два раза больше, чем учебного.

### 4. Применение автокорректора


Протестировала несколько автокорректоров: SpellChecker, Hunspell, DeepPavlov

#### **SpellChecker**

**Автокоррекция литературного текста**

In [34]:
# Инициализация автокорректора для русского языка
spell = SpellChecker(language='ru')

Так как текст литературный, в нем встречаются слова, которые используются в разговорной речи, придуманные автором слова, сложные слова. Они не распознаются стандартными программами для проверки орфографии. Эти слова я добавила в словарь, чтобы автокорретор не учитывал их как ошибку.

In [35]:
# Добавляем слова в словарь (чтобы они не исправлялись)
custom_words = [
    "усматривались", "возгоняли", "гореутолителей", "злободробитель",
    "отомкнул", "пятидесятиголовые", "взревывания", "противослоновый",
    "соплеменными", "тралением", "валявшуюся", "самовозгореться",
    "самовозгорелось", "немножко", "томительными", "нужностью",
    "девятимерном", "магопространстве", "экономного", "сигаретками",
    "исцарапанным", "вылуплялись", "компрэне"
    ]
spell.word_frequency.load_words(custom_words)

In [36]:
# Функция для автокорректировки текста
def correct_text_sc(text):
    words = text.split()
    corrected_words = [
        spell.correction(word) if word not in spell.word_frequency else word
        for word in words
    ]
    # Проверка на None и замена на исходное слово
    corrected_words = [word if corrected_word is None else corrected_word
                       for word, corrected_word in zip(words, corrected_words)]

    return " ".join(corrected_words)

In [37]:
# автокоррекция текста без обработки тире и знаков препинания
correct_text_spellcheck = correct_text_sc(recognized_text)
print(correct_text_spellcheck[0:1000])

по-видимому, я вестибюля. я заплесневелых глаз белое держали кости прикованных скелетов, где-то верно канала вода я наша между колонна я неестественных поза торчать статуи я ржавый латах, справа от входа у стены громоздились обломки древние идолов, наверху этой куча торчать рисовые ноги я сапогах. я почерневших портрет под потолок строго выбрали маститые старый я их лица усматривались знакомые черт флора Симеоновича, товарищ диана знакомо и других мастеров. Весь этот архаический храм надлежало давным-давно выбросить прорубить я стена окна и установить трубку немного света но все было заприходовано, заинвентаризовано и лично местом Матвеевичем я разбазариванию запрещено На капителях колонна и я лабиринт исполинской люстры, свисающей я почерневшего потолка, шуршали нетопыри и летучие собаки я ними может Матвеевич боролся Он плевал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они или тысячами, но возрождались десятки тысяч Они мутировали, среди них появились поищи и

Видно, что в стандартном словаре нет таких простых предлогов как "с" и "в".


**Подсчет метрик**

In [92]:
wer_corr_sc, cer_corr_sc = metrics_wer_cer(etalon_text, correct_text_spellcheck)
print(f"WER: {wer_corr_sc}")
print(f"CER: {cer_corr_sc}")

WER: 0.364
CER: 0.062


In [39]:
# посмотрим результат коррекции
print(correct_text_spellcheck[0:1000])
print(etalon_text[0:1000])

по-видимому, я вестибюля. я заплесневелых глаз белое держали кости прикованных скелетов, где-то верно канала вода я наша между колонна я неестественных поза торчать статуи я ржавый латах, справа от входа у стены громоздились обломки древние идолов, наверху этой куча торчать рисовые ноги я сапогах. я почерневших портрет под потолок строго выбрали маститые старый я их лица усматривались знакомые черт флора Симеоновича, товарищ диана знакомо и других мастеров. Весь этот архаический храм надлежало давным-давно выбросить прорубить я стена окна и установить трубку немного света но все было заприходовано, заинвентаризовано и лично местом Матвеевичем я разбазариванию запрещено На капителях колонна и я лабиринт исполинской люстры, свисающей я почерневшего потолка, шуршали нетопыри и летучие собаки я ними может Матвеевич боролся Он плевал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они или тысячами, но возрождались десятки тысяч Они мутировали, среди них появились поищи и

Уже видно, как корректор неудачно обработал часть слов.

Метрики сильно ухудшились. Предполагаю, что если очистить текст от лишних знаков препинания и тире, то метрики будут лучше.

**Улучшения**

In [62]:
def cleaned_text(text):
    # Регулярное выражение для удаления знаков препинания и тире
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    # Удаление тире, включая длинные тире (—)
    cleaned_text = re.sub(r'[-—–]', ' ', cleaned_text)
    # Удаляет лишние пробелы из текста, оставляя только один
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [63]:
# Очищенный распознанный текст
cleaned_recognized_text = cleaned_text(recognized_text)
print(cleaned_recognized_text[0:1000])

по видимому с вестибюля В заплесневелых углах белесо мерцали кости прикованных скелетов где то мерно капала вода в нишах между колоннами в неестественных позах торчали статуи в ржавых латах справа от входа у стены громоздились обломки древних идолов наверху этой кучи торчали гипсовые ноги в сапогах С почерневших портретов под потолком строго взирали маститые старцы в их лицах усматривались знакомые черты Федора Симеоновича товарища Жиана Жиакомо и других мастеров Весь этот архаический хлам надлежало давным давно выбросить прорубить в стенах окна и установить трубки дневного света но все было заприходовано заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено На капителях колонн и в лабиринтах исполинской люстры свисающей с почерневшего потолка шуршали нетопыри и летучие собаки С ними Модест Матвеевич боролся Он поливал их скипидаром и креозотом опылял дустом опрыскивал гексахлораном они гибли тысячами но возрождались десятками тысяч Они мутировали среди них появляли

In [64]:
# Очищенный эталонный текст
cleaned_etalon_text = cleaned_text(etalon_text)
# удалим пробел в начале строки
cleaned_etalon_text = cleaned_etalon_text.lstrip()
print(cleaned_etalon_text[0:1000])

по видимому с вестибюля В заплесневелых углах белесо мерцали кости прикованных скелетов где то мерно капала вода в нишах между колоннами в неестественных позах торчали статуи в ржавых латах справа от входа у стены громоздились обломки древних идолов наверху этой кучи торчали гипсовые ноги в сапогах С почерневших портретов под потолком строго взирали маститые старцы в их лицах усматривались знакомые черты Федора Симеоновича товарища Жиана Жиакомо и других мастеров Весь этот архаический хлам надлежало давным давно выбросить прорубить в стенах окна и установить трубки дневного света но все было заприходовано заинвентаризировано и лично Модестом Матвеевичем к разбазариванию запрещено На капителях колонн и в лабиринтах исполинской люстры свисающей с почерневшего потолка шуршали нетопыри и летучие собаки С ними Модест Матвеевич боролся Он поливал их скипидаром и креозотом опылял дустом опрыскивал гексахлораном они гибли тысячами но возрождались десятками тысяч Они мутировали среди них появля

In [65]:
custom_words.extend(['с', 'в']) # добавим предлоги 'с', 'в'

In [66]:
spell.word_frequency.load_words(custom_words) # обновляем словарь

In [67]:
# автокорректировка распознанного и очищенного текста
correct_text_sc_clean = correct_text_sc(cleaned_recognized_text)
print(correct_text_sc_clean[0:1000])

по видимо с вестибюля В заплесневелых глаз белое держали кости прикованных скелет где то верно канала вода в наша между колонна в неестественных поза торчать статуи в ржавый запах справа от входа у стены громоздились обломки древние долой наверху этой куча торчать рисовые ноги в сапоги С почерневших портрет под потолок строго выбрали маститые старый в их лица усматривались знакомые черт флора Симеоновича товарищ диана знакомо и других мастер Весь этот архаический храм надлежало давным давно выбросить прорубить в стена окна и установить трубку немного света но все было заприходовано заинвентаризовано и лично местом Матвеевичем я разбазариванию запрещено На капителях колонна и в лабиринт исполинской сестры свисающей с почерневшего посылка шуршали нетопыри и летучие собаки С ними может Матвеевич боролся Он плевал их скипидаром и креозотом опылял пустой опрыскивал гексахлораном они или тысячи но возрождались десятки тысяч Они мутировали среди них появились поищи и разговаривающие штаны пот

In [68]:
# считаем метрики для очищенного текста
wer_corr_sc_clean, cer_corr_sc_clean = metrics_wer_cer(cleaned_etalon_text,
                                                       correct_text_sc_clean)
print(f"WER: {wer_corr_sc_clean}")
print(f"CER: {cer_corr_sc_clean}")

WER: 0.241
CER: 0.061


**Результат:**

Метрики стали лучше, однако видно, что в стандартных словарях нет таких простых предлогов как "с" и автокорректор исправляет формы слов, которые не используются без тире, например, "по-видимому" слово "видимому" не используется без тире и оно было исправлено на "видимо".

Очевидно, что этот автокорректор ухудшает текст, так как в целом метрики стали значительно хуже.

Ниже визуально вывела на экран слова, которые заменяет автокорректор и на какие слова заменяет. Вывод сократила до 100 ошибко, так как работает довольно долго.

In [69]:
# извлечём слова, в которых
# предположительно находится ошибка и сохраним их в mistakes:
mistakes = spell.unknown(cleaned_recognized_text.split())
for mistake in list(mistakes)[:100]:
    print(f'Ошибка: {mistake} Правильно: {spell.correction(mistake)}')

Ошибка: бесконечности Правильно: бесконечность
Ошибка: диковинных Правильно: None
Ошибка: металлическое Правильно: металлические
Ошибка: постройке Правильно: постройте
Ошибка: предсказаний Правильно: предсказание
Ошибка: молотками Правильно: None
Ошибка: сожран Правильно: сохрани
Ошибка: тихон Правильно: тихо
Ошибка: толстыми Правильно: толстый
Ошибка: отдельной Правильно: отдельно
Ошибка: сторукие Правильно: None
Ошибка: мутировали Правильно: None
Ошибка: нетопыри Правильно: None
Ошибка: прошелся Правильно: прошел
Ошибка: потупился Правильно: получился
Ошибка: китайского Правильно: китайское
Ошибка: рентабельности Правильно: None
Ошибка: двутавровыми Правильно: None
Ошибка: выходе Правильно: выходи
Ошибка: унылая Правильно: None
Ошибка: наполеона Правильно: наполеон
Ошибка: административно Правильно: None
Ошибка: гекатонхейры Правильно: None
Ошибка: вылечивались Правильно: None
Ошибка: ифрита Правильно: рита
Ошибка: питались Правильно: пытались
Ошибка: числил Правильно: число
Ошибка: 

Посмотрела какие конкретно исправлялись ошибки и на какие слова. В словаре НЕ учтены все формы слов и некоторые слова написанные правильно, но отсутствующие в словаре, тоже исправлялись на неправильные. Например, "людоед" заменено на "люди", "ватник" на "ватикан" и т.п. Видимо поэтому метрики возросли. Проверку закоментировала, так как она довольно долго работает.

**Автокоррекция текста из учебника**

In [70]:
# автокоррекция текста без обработки тире и знаков препинания
correct_recog_sc_tgp = correct_text_sc(recognized_text_tgp)
print(correct_recog_sc_tgp[0:1000])

субъект правопонимания всегда выступает конкретные человек например а гражданин обладающий минимальный правовым кругозором, столкнувшийся с проблема права вообще я юрист-профессионал, имеющий достаточно запас знание о правда способный применить и толковать правое норман вы ученый человек с абстрактным мышлением, занимающийся изучение права обладающий самой исторический и современные знание способный я интерпретации не только норма но и принцип права владеющий определенно методологией исследования Правопонимание всегда субъективно, оригинальное хотя представления о права могут совпадает у группы лиц и у целых слово классов. объект правопонимания могут быть право в планетарном масштабе, право конкретно общества отрасль, институт права отдельные правое норман При этом знания об отдельные структурных элемент экстраполируются на право в целом важно познавательную загрузка здесь несут среда и взаимодействующие с право общественные явление Содержание правопонимания составляют знания субъект о

In [71]:
# считаем метрики
wer_sc_tgp, cer_sc_tgp = metrics_wer_cer(etalon_text_tgp,
                                         correct_recog_sc_tgp)
print(f"WER: {wer_sc_tgp}")
print(f"CER: {cer_sc_tgp}")

WER: 0.328
CER: 0.047


In [72]:
# Очищаем тексты
clean_recog_text_tgp = cleaned_text(recognized_text_tgp)
clean_etalon_text_tgp = cleaned_text(etalon_text_tgp)
clean_etalon_text_tgp = clean_etalon_text_tgp.lstrip() # удалим пробел в начале строки
print(clean_recog_text_tgp[0:1000])
print(clean_etalon_text_tgp[0:1000])

Субъектом правопонимания всегда выступает конкретный человек например а гражданин обладающий минимальным правовым кругозором столкнувшийся с проблемами права вообще 6 юрист профессионал имеющий достаточный запас знаний о праве способный применять и толковать правовые нормы в ученый человек с абстрактным мышлением занимающийся изучением права обладающий суммой исторических и современных знаний способный к интерпретации не только норм но и принципов права владеющий определенной методологией исследования Правопонимание всегда субъективно оригинально хотя представления о праве могут совпадать у группы лиц и у целых слоев классов Объектом правопонимания могут быть право в планетарном масштабе право конкретного общества отрасль институт права отдельные правовые нормы При этом знания об отдельных структурных элементах экстраполируются на право в целом Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления Содержание правопонимания составляют знания 

In [73]:
# автокорректировка распознанного и очищенного текста
correct_recog_sc_tgp_clean = correct_text_sc(clean_recog_text_tgp)
print(correct_recog_sc_tgp_clean[0:1000])

субъект правопонимания всегда выступает конкретные человек например а гражданин обладающий минимальный правовым кругозором столкнувшийся с проблема права вообще 6 юрист профессионал имеющий достаточно запас знание о права способный применить и толковать правое норма в ученый человек с абстрактным мышлением занимающийся изучение права обладающий самой исторический и современные знание способный я интерпретации не только нора но и принцип права владеющий определенно методологией исследования Правопонимание всегда субъективно оригинальное хотя представления о права могут совпадает у группы лиц и у целых слов классно объект правопонимания могут быть право в планетарном масштабе право конкретно общества страсть институт права отдельные правое норма При этом знания об отдельные структурных элемент экстраполируются на право в целом важно познавательную загрузка здесь несут среда и взаимодействующие с право общественные явление Содержание правопонимания составляют знания субъект о его права и 

In [74]:
# считаем метрики для очищенного текста
wer_sc_clean_tgp, cer_sc_clean_tgp = metrics_wer_cer(
    clean_etalon_text_tgp,
    correct_recog_sc_tgp_clean
    )
print(f"WER: {wer_sc_clean_tgp}")
print(f"CER: {cer_sc_clean_tgp}")

WER: 0.257
CER: 0.052


Видно, что очистка текста улучшила метрики, однако все равно 24-25% - это плохие результаты.

Литературный текст до улучшения имел метрики: WER: 36,4%, CER: 6,2%

Литературный текст после улучшения: WER: 24,1%, CER: 6,1%

Текст из учебника до улучшения имел метрики: WER: 32,8%, CER: 4,7%

Текст из учебника после улучшения: WER: 25,7%, CER: 5,2%

В целом, автокорректор SpellChecker дал плохой результат.

#### **HunSpell**

In [75]:
# Инициализация HunSpell с русским словарем
hunspell = hunspell.HunSpell('/usr/share/hunspell/ru_RU.dic', '/usr/share/hunspell/ru_RU.aff')

In [76]:
def correct_text_hs(text):
    words = text.split()
    corrected_words = []

    for word in words:
        if not hunspell.spell(word):
            suggestions = hunspell.suggest(word)
            if suggestions:
                corrected_words.append(suggestions[0])  # Используем первое предложение
            else:
                corrected_words.append(word)  # Если нет предложений, оставляем слово как есть
        else:
            corrected_words.append(word)

    return ' '.join(corrected_words)

**Литературный текст**

In [77]:
# автокоррекция текста без обработки тире и знаков препинания
correct_reg_text_hs = correct_text_hs(recognized_text)
print(correct_reg_text_hs[0:1000])

по-видимому с вестибюля. В заплесневелых углах белесое мерцали кости прикованных скелетов где-то мерно капала вода в нишах между колоннами в неестественных позах торчали статуи в ржавых латах справа от входа у стены громоздились обломки древних идолов наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы в их лицах усматривались знакомые черты Федора Самсоновичем товарища Жана Знакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить прорубить в стенах окна и установить трубки дневного света но все было заприходовано заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено. На капителях колонн и в лабиринтах исполинской люстры свисающей с почерневшего потолка шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом опылял дустом опрыскивал гексахлораном они гибли тысячами но возрождались десятками тысяч. Они мутировали среди них 

In [78]:
wer_corr_hs, cer_corr_hs = metrics_wer_cer(etalon_text,
                                           correct_reg_text_hs)
print(f"WER: {wer_corr_hs}")
print(f"CER: {cer_corr_hs}")

WER: 0.191
CER: 0.026


In [79]:
# сравним визуально оба текста
print(correct_reg_text_hs[0:1000])
print(etalon_text[0:1000])

по-видимому с вестибюля. В заплесневелых углах белесое мерцали кости прикованных скелетов где-то мерно капала вода в нишах между колоннами в неестественных позах торчали статуи в ржавых латах справа от входа у стены громоздились обломки древних идолов наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы в их лицах усматривались знакомые черты Федора Самсоновичем товарища Жана Знакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить прорубить в стенах окна и установить трубки дневного света но все было заприходовано заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено. На капителях колонн и в лабиринтах исполинской люстры свисающей с почерневшего потолка шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом опылял дустом опрыскивал гексахлораном они гибли тысячами но возрождались десятками тысяч. Они мутировали среди них 

При визуальном сравнении видно, что в распознанном и откорректорованном корректор удалил запятые . Уберем их в эталонном тексте.

**Улучшения:**

In [81]:
# Удаление запятых
etalon_text_corr_hs = etalon_text.replace(',', '')
print(etalon_text_corr_hs[0:1000])

﻿по-видимому с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов где-то мерно капала вода в нишах между колоннами в неестественных позах торчали статуи в ржавых латах справа от входа у стены громоздились обломки древних идолов наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы в их лицах усматривались знакомые черты Федора Симеоновича товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить прорубить в стенах окна и установить трубки дневного света но все было заприходовано заинвентаризировано и лично Модестом Матвеевичем к разбазариванию запрещено. На капителях колонн и в лабиринтах исполинской люстры свисающей с почерневшего потолка шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом опылял дустом опрыскивал гексахлораном они гибли тысячами но возрождались десятками тысяч. Они мутировали среди ни

In [82]:
wer_corr_hs_et_corr, cer_corr_hs_et_corr = metrics_wer_cer(
    etalon_text_corr_hs,
    correct_reg_text_hs
    )
print(f"WER: {wer_corr_hs_et_corr}")
print(f"CER: {cer_corr_hs_et_corr}")

WER: 0.107
CER: 0.026


Метрики уменьшились, но не значительно. Предполагаю, что дело опять в словаре.

**Текст из учебника**

In [83]:
# автокоррекция текста без обработки тире и знаков препинания
correct_text_hs_tgp = correct_text_hs(recognized_text_tgp)
print(correct_text_hs_tgp[0:1000])

Субъектом право понимания всегда выступает конкретный человек например а гражданин обладающий минимальным правовым кругозором столкнувшийся с проблемами права вообще 6) юрист-профессионал имеющий достаточный запас знаний о праве способный применять и толковать правовые нормы в ученый человек с абстрактным мышлением занимающийся изучением права обладающий суммой исторических и современных знаний способный к интерпретации не только норм но и принципов права владеющий определенной методологией исследования. Право понимание всегда субъективно оригинально хотя представления о праве могут совпадать у группы лиц и у целых слоев классов. Объектом право понимания могут быть право в планетарном масштабе право конкретного общества отрасль институт права отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. Содержание право понимания составля

In [84]:
wer_corr_hs_tgp, cer_corr_hs_tgp = metrics_wer_cer(etalon_text_tgp,
                                                   correct_text_hs_tgp)
print(f"WER: {wer_corr_hs_tgp}")
print(f"CER: {cer_corr_hs_tgp}")

WER: 0.194
CER: 0.014


In [86]:
# Удаление запятых
etalon_text_tgp_hs = etalon_text_tgp.replace(',', '')
print(etalon_text_tgp_hs[0:1000])

﻿Субъектом правопонимания всегда выступает конкретный человек например: а) гражданин обладающий минимальным правовым кругозором столкнувшийся с проблемами права вообще; б) юрист-профессионал имеющий достаточный запас знаний о праве способный применять и толковать правовые нормы; в) ученый человек с абстрактным мышлением занимающийся изучением права обладающий суммой исторических и современных знаний способный к интерпретации не только норм но и принципов права владеющий определенной методологией исследования. Правопонимание всегда субъективно оригинально хотя представления о праве могут совпадать у группы лиц и у целых слоев классов. Объектом правопонимания могут быть право в планетарном масштабе право конкретного общества отрасль институт права отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. Содержание правопонимания состав

In [88]:
wer_corr_hs_tgp_2, cer_corr_hs_tgp_2 = metrics_wer_cer(etalon_text_tgp_hs,
                                                       correct_text_hs_tgp)
print(f"WER: {wer_corr_hs_tgp_2}")
print(f"CER: {cer_corr_hs_tgp_2}")

WER: 0.103
CER: 0.014


Литературный текст до улучшения имел метрики: WER: 19,1%, CER: 2,6%

Литературный текст после улучшения: WER: 10,7%, CER: 2,6%

Текст из учебника имеет метрики: WER: 19,4$, CER: 1,4%

Текст из учебника после улучшения: WER: 10,3$, CER: 1,4%

Автокорректор HunSpell показал себя лучше, чем предыдущий, но все равно не достаточно хорошо.

#### **DeepPavlov**

In [ ]:
# Загружаем предобученную модель коррекции ошибок
model = build_model(configs.spelling_correction.levenshtein_corrector_ru, download=True)

2025-02-01 19:25:27.944 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz download because of matching hashes
2025-02-01 19:25:29.154 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz download because of matching hashes


In [ ]:
# Функция для исправления текста с учетом пользовательского словаря
def correct_text_dp(text):
    words = text.split()
    corrected_words = [
        word if word in custom_words else model([word])[0]  # Не исправляем слова из словаря
        for word in words
    ]
    return " ".join(corrected_words)

**Литературный текст**

In [ ]:
# автокоррекция текста без обработки тире и знаков препинания
correct_text_deeppavlov = correct_text_dp(recognized_text)
print(correct_text_deeppavlov[0:1000])

по-видимому, с вестибюля. в заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. с почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты федора симеоновича, товарища лиана жиакомо и других мастеров. весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично модестом матвеевичем к разбазариванию запрещено. на капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. с ними модест матвеевич боролся. он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятками тысяч. они мутиро

In [ ]:
wer_corr_dp, cer_corr_dp = metrics_wer_cer(etalon_text,
                                           correct_text_deeppavlov)
print(f"WER: {wer_corr_dp}")
print(f"CER: {cer_corr_dp}")

WER: 0.177
CER: 0.005


**Текст из учебника**

In [ ]:
# автокоррекция текста без обработки тире и знаков препинания
correct_text_dp_tgp = correct_text_dp(recognized_text_tgp)
print(correct_text_dp_tgp[0:1000])

субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов. объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. при этом знания об отдельных структурных элементах экстраполируются на право в целом. важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. содержание прав

In [ ]:
wer_corr_dp_tgp, cer_corr_dp_tgp = metrics_wer_cer(etalon_text_tgp,
                                                   correct_text_dp_tgp)
print(f"WER: {wer_corr_dp_tgp}")
print(f"CER: {cer_corr_dp_tgp}")

WER: 0.099
CER: 0.003


Литературный текст имеет метрики: WER: 17,7%, CER: 0,5%

Текст из учебника имеет метрики: WER: 9,9%, CER: 0,3%

### 5. Анализ и выводы

**Сравнение автокорректоров по метрикам**

In [ ]:
df = [[wer_recognized, cer_recognized],
      [wer_corr_sc_clean, cer_corr_sc_clean],
      [wer_corr_hs_et_corr, cer_corr_hs_et_corr],
      [wer_corr_dp, cer_corr_dp]]
autocorrect = ["Without autocorrect", "SpellChecker", "HunSpell", "DeepPavlov"]

In [ ]:
print("Таблица сравнения автокорректоров для литературного текста")
pd.DataFrame(data=df, index=autocorrect,
             columns=["WER", "CER"])

Таблица сравнения автокорректоров для литературного текста


,WER,CER
Without autocorrect,0.046,0.004
SpellChecker,0.322,0.055
HunSpell,0.107,0.027
DeepPavlov,0.177,0.005


In [ ]:
df_tgp = [[wer_recognized_tgp, cer_recognized_tgp],
          [wer_sc_tgp, cer_sc_tgp],
          [wer_corr_hs_tgp, cer_corr_hs_tgp],
          [wer_corr_dp_tgp, cer_corr_dp_tgp]]

In [ ]:
print("Таблица сравнения автокорректоров для учебного текста")
pd.DataFrame(data=df_tgp, index=autocorrect,
             columns=["WER", "CER"])

Таблица сравнения автокорректоров для учебного текста


,WER,CER
Without autocorrect,0.023,0.002
SpellChecker,0.328,0.047
HunSpell,0.194,0.016
DeepPavlov,0.099,0.003


**Выводы:**

Подготовила текст - отрывок из книги братьев Стругацких "Понедельник начинается в субботу" в формате jpg. Литературный текст с небольшим количеством цифр.
Так же подготовила этот же текст в файле в формате txt как эталонный.
Из слов имеющих формы, котоые отсутствуют в стандартных словарях для проверки орфографии собран небольшой словарь.

Текст в формате jpg был распозанан моделью Tesseract и собран в единый файл.

Далее сравнила распознанный текст с эталонным и вычислила метрики. Метрики для  текстов без применения автокоррекции равны WER: 4,6% и CER: 0,4%.

Применила автокорректоры:
- SpellChecker
- HunSpell
- DeepPavlov

Все автокорректоры увеличивают ошибку. Лучшие метрики показал текст без использования автокорректоров. Могу предположить, что проблема в отсутсвии полного встроенного словаря русского языка, который влючает все морфологические формы и специфические слова.

В целом, текст из учебника показал метрики чуть лучше, однако они тоже не позазали улучшений относительно метрик без автокоррекции. Скорее всего слова в учебном тексте более распространенные и находятся в стандартных словарях чуть чаще.

Лучше всех себя показал DeepPavlov, его метрики лучше остальных автокорректоров WER - 9,9% CER - 0,3%. Однако, все равно метрики без автокоррекции заметно лучше.  